Simple ridge baseline with GroupKFold CV split by investment id. The last 2 mio rows are used for training due to memory restrictions. Investment id features are replaced with the corresponding target means.

Dataset is taken from: https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr as p

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_pickle('../input/ump-train-picklefile/train.pkl')[-2000000:]
kfold = GroupKFold(n_splits=5)
feats = [f for f in train.columns if f not in ['time_id', 'row_id', 'target']]
target = 'target'

In [ ]:
inv_means = train.groupby('investment_id')['target'].mean()
train['investment_id'] = train['investment_id'].map(inv_means)

In [ ]:
train[feats]

In [ ]:
fold_scores = []
models = []

for fold, (trn_idx, val_idx) in enumerate(kfold.split(train[feats], train[target], groups=train['time_id'])):
    xtrain, ytrain = train.iloc[trn_idx][feats], train.iloc[trn_idx][target]
    xval, yval = train.iloc[val_idx][feats], train.iloc[val_idx][target]
    
    model = Ridge(alpha=100000, normalize=False)
    model.fit(xtrain, ytrain)
    val_preds = model.predict(xval)
    
    score = p(val_preds, yval)[0]
    print(f"Fold {fold + 1}: {score}")
    fold_scores.append(score)
    models.append(model)

In [ ]:
print(f"Overall score: {np.mean(fold_scores)}")

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[feats]
    test_df['investment_id'] = test_df['investment_id'].map(inv_means).fillna(0)
    
    pred_0 = models[0].predict(test_df)
    pred_1 = models[1].predict(test_df)
    pred_2 = models[2].predict(test_df)
    pred_3 = models[3].predict(test_df)
    pred_4 = models[4].predict(test_df)
    pred = (pred_0 + pred_1 + pred_2 + pred_3 + pred_4) / 5
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df) 